In [1]:
import xgboost
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error

from imblearn.under_sampling import RandomUnderSampler

In [3]:
df = pd.read_csv("./datasets/used_cars_Pakistan.csv", index_col = 0)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './datasets/used_cars_Pakistan.csv'

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86120 entries, 0 to 86119
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make                 86120 non-null  object 
 1   model                86120 non-null  object 
 2   city                 86120 non-null  object 
 3   year                 86120 non-null  int64  
 4   mileage              86120 non-null  int64  
 5   Engine_displacement  85931 non-null  float64
 6   Battery              189 non-null    float64
 7   Price_Rs             86120 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 5.9+ MB


In [5]:
df['is_electric'] = (df['Engine_displacement'].isna()) & (df['Battery']> 0)
df['Engine_displacement'] = np.where(df['is_electric'] == True, 0.0, df['Engine_displacement'])
df['Battery'] = np.where(df['is_electric'] == False, 0.0, df['Battery'])

In [6]:
for col in list(df.select_dtypes(['object']).columns):
    df[col] = df[col].astype('category')

In [7]:
X = df.drop('Price_Rs', axis = 1)
y = pd.DataFrame(df['Price_Rs'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=11)

In [10]:
parameters = {'n_estimators':[200, 300, 400, 500, 600, 700], 'max_depth': np.arange(2,6), 'learning_rate':[0.02, 0.05, 0.1, 0.15, 0.2, 0.25]}
xgb = xgboost.XGBRegressor(enable_categorical = True, random_state = 11)

In [11]:
gs = GridSearchCV(xgb, parameters, verbose = 10, scoring = 'neg_mean_squared_error')
gs.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 1/5; 1/144] START learning_rate=0.02, max_depth=2, n_estimators=200.........
[CV 1/5; 1/144] END learning_rate=0.02, max_depth=2, n_estimators=200;, score=-9018436127846.420 total time=   0.9s
[CV 2/5; 1/144] START learning_rate=0.02, max_depth=2, n_estimators=200.........
[CV 2/5; 1/144] END learning_rate=0.02, max_depth=2, n_estimators=200;, score=-8978216182909.408 total time=   0.8s
[CV 3/5; 1/144] START learning_rate=0.02, max_depth=2, n_estimators=200.........
[CV 3/5; 1/144] END learning_rate=0.02, max_depth=2, n_estimators=200;, score=-8212810649637.274 total time=   0.7s
[CV 4/5; 1/144] START learning_rate=0.02, max_depth=2, n_estimators=200.........
[CV 4/5; 1/144] END learning_rate=0.02, max_depth=2, n_estimators=200;, score=-343387568154180.188 total time=   0.7s
[CV 5/5; 1/144] START learning_rate=0.02, max_depth=2, n_estimators=200.........
[CV 5/5; 1/144] END learning_rate=0.02, max_depth=2, n_estimators=

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=True, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=11, ...),
             param_grid={'learning_rate': [0.02, 0.05, 0.1, 0.15, 0.2, 0.25],
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': [200, 300, 400, 500, 600, 700]},
             scoring='neg_mean_squared_error', verbose=10)

In [12]:
print(gs.best_params_, gs.best_score_)

{'learning_rate': 0.02, 'max_depth': 2, 'n_estimators': 400} -75114456679236.12


In [20]:
best_model = gs.best_estimator_
y_pred = best_model.predict(X_test)
print(mean_squared_error(y_test, y_pred))

14314229408988.332
